# Azure AI Agent with Azure AI Search

This tutorial shows how to:
1) Create an Azure AI client, 2) Configure the Azure AI Search tool, 3) Create an agent, 4) Ask a question against your search index, and 5) Clean up resources.

## Prerequisites
- An Azure AI Project with access to your Azure AI Search connection.
- An Azure OpenAI model deployment name in your project.
- Environment variables set:
  - `PROJECT_ENDPOINT` (Azure AI Project endpoint)
  - `MODEL_DEPLOYMENT_NAME` (your model deployment name)
  - Optional: `search_index_name` (defaults to `finance-pro` from your repo)

You can set environment variables (PowerShell):
```pwsh
$env:PROJECT_ENDPOINT = "https://<your-project-endpoint>"
$env:MODEL_DEPLOYMENT_NAME = "<your-model-deployment-name>"
$env:search_index_name = "finance-pro"
```

In [ ]:
%pip install -q azure-ai-projects azure-ai-agents azure-identity python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Load .env if present
load_dotenv()

PROJECT_ENDPOINT = os.getenv("PROJECT_ENDPOINT")
MODEL_DEPLOYMENT_NAME = os.getenv("MODEL_DEPLOYMENT_NAME")
INDEX_NAME = os.getenv("search_index_name", "finance-pro")

missing = [name for name, val in {
    "PROJECT_ENDPOINT": PROJECT_ENDPOINT,
    "MODEL_DEPLOYMENT_NAME": MODEL_DEPLOYMENT_NAME
}.items() if not val]
if missing:
    raise RuntimeError(f"Missing required environment variables: {', '.join(missing)}")

print("Environment OK. Using index:", INDEX_NAME)

## 1) Create an Azure AI Client

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(exclude_interactive_browser_credential=False),
)
project_client

## 2) Configure the Azure AI Search tool
Uses the default Azure AI Search connection configured in your Azure AI Project.

In [ ]:
from azure.ai.agents.models import AzureAISearchTool, AzureAISearchQueryType
from azure.ai.projects.models import ConnectionType

# Get the default Azure AI Search connection ID from the project
try:
    azure_ai_conn_id = project_client.connections.get_default(ConnectionType.AZURE_AI_SEARCH).id
except Exception as ex:
    raise RuntimeError("Failed to get default Azure AI Search connection ID")

ai_search = AzureAISearchTool(
    index_connection_id=azure_ai_conn_id,
    index_name=INDEX_NAME,
    query_type=AzureAISearchQueryType.SIMPLE,  # SIMPLE or SEMANTIC
    top_k=3,
    filter=""
)
ai_search

## 3) Create an agent with the Azure AI Search tool

In [ ]:
agent = project_client.agents.create_agent(
    model=MODEL_DEPLOYMENT_NAME,
    name="my-agent",
    instructions="You are a helpful agent that answers questions using the Azure AI Search tool.",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources,
)
print(f"Created agent, ID: {agent.id}")

## 4) Ask the agent questions about data in the index

In [ ]:
from azure.ai.agents.models import MessageRole, ListSortOrder

# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Send a message to the thread
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content="Whow do you value a company??",
)
print(f"Created message, ID: {message.id}")

# Create and process a run
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch messages
messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for m in messages:
    print(m.content)

## 5) Clean up resources (optional)

In [ ]:
project_client.agents.delete_agent(agent.id)
print("Deleted agent")